In [62]:
import tools
import importlib
import memory
importlib.reload(tools)
importlib.reload(memory)

<module 'memory' from '/home/c0ff3k1ll3r/Desktop/Thesis/AI_Agent_for_honeypot_operations/HoneypotAgentApp/MultiAgentArchitecture/memory.py'>

In [63]:
alerts = await tools.get_fast_alerts(time_window=5)
alerts

{'security_events': {'alerts': [],
  'count': 0,
  'time_window_minutes': 5,
  'timestamp': '2025-08-02T12:18:29.127308'}}

In [64]:
tools.get_docker_containers()

{'honeypot_config': [{'id': '3c926ef3439b',
   'name': 'gitlab-proxy',
   'image': 'nginx:alpine',
   'status': 'running',
   'created': '2025-08-02T08:03:02.827694162Z',
   'ports': ['80/tcp'],
   'ip_address': '172.20.0.7'},
  {'id': '08402f6057a7',
   'name': 'cve-2021-22205-gitlab-1',
   'image': 'vulhub/gitlab:13.10.1',
   'status': 'running',
   'created': '2025-08-02T08:03:02.803271897Z',
   'ports': ['22/tcp', '443/tcp', '80/tcp', '8080/tcp'],
   'ip_address': '172.20.0.6'},
  {'id': '137d39aec474',
   'name': 's2-057-struts2-1',
   'image': 'vulhub/struts2:2.3.34-showcase',
   'status': 'running',
   'created': '2025-08-02T08:03:02.443227994Z',
   'ports': ['8080/tcp'],
   'ip_address': '172.20.0.3'},
  {'id': '8fd1379e4888',
   'name': 'unauthorized-rce-docker-1',
   'image': 'unauthorized-rce-docker:latest',
   'status': 'running',
   'created': '2025-08-02T08:03:02.160808847Z',
   'ports': ['2375/tcp', '2376/tcp'],
   'ip_address': '172.20.0.2'}]}

In [84]:
print(tools.check_services_health())
rules = await tools.get_firewall_rules()
print(rules['firewall_config']['data']['rules'])

{'firewall_status': 'up', 'suricata_status': 'up'}
Chain FORWARD (policy DROP)
num  target     prot opt source               destination         
1    ACCEPT     all  --  0.0.0.0/0            0.0.0.0/0            state RELATED,ESTABLISHED
2    ACCEPT     icmp --  0.0.0.0/0            0.0.0.0/0           
3    ACCEPT     all  --  172.20.0.0/24        172.20.0.0/24       
4    LOG        all  --  0.0.0.0/0            0.0.0.0/0            LOG flags 0 level 4 prefix "FIREWALL-DROP: "



In [83]:
await tools.remove_firewall_rule([1])

INFO:tools:Removing firewall rules: [1]


{'rules_removed_current_epoch': ['REMOVED rule #1:    ACCEPT     tcp  --  192.168.100.0/24     172.20.0.7']}

In [12]:
honeypot_ip = '172.20.0.2' 
attacker_ip = '192.168.100.2'

await tools.add_allow_rule(source_ip=attacker_ip, dest_ip=honeypot_ip)
await tools.add_allow_rule(source_ip=honeypot_ip, dest_ip=attacker_ip)

INFO:tools:Adding allow rule: ALLOW 192.168.100.2 -> 172.20.0.2)
INFO:tools:Adding allow rule: ALLOW 172.20.0.2 -> 192.168.100.2)


{'rules_added_current_epoch': 'ALLOW 172.20.0.2 -> 192.168.100.2)'}